In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import json

In [6]:
driver = webdriver.Chrome()
driver.get('http://www.portaltransparencia.gov.br/')

# Acessa pagina dos servidores
driver.find_element_by_id('servidores-card').click()
aTagsInLi = driver.find_elements_by_css_selector('li a')
for a in aTagsInLi:
    if a.get_attribute('href') == 'http://www.portaltransparencia.gov.br/servidores/lista-consultas':
        a.click()
        break

# Seleciona consulta por vinculo 
driver.find_element_by_xpath('/html/body/main/div[2]/div/div/ul/li[2]/a').click()

#Seleciona os servidores do tipo militar e civil
driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/button').click()

try:
    militar = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[3]/a/label/input'))
    )
    civil = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[4]/a/label/input'))
    )
    
finally:
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[3]/a/label/input').click()
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[4]/a/label/input').click()
    driver.find_element_by_xpath('//*[@id="id-box-filtro"]/div/div/ul/li[10]/div/div/div/div[2]/div[2]/ul/li[2]/input').click()

driver.find_element_by_class_name('botao__gera_paginacao_completa').click()


## Retirando dados de cada servidor

try:
    paginacao_completa = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'td a'))
    )
    
finally:
    aTagsInTd = driver.find_elements_by_css_selector('td a')


pagina_servidores = []
for a in aTagsInTd:
    pagina_servidores.append(a.get_attribute('href'))
    
    
for pagina in pagina_servidores:
    print(pagina)
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(pagina)
    
    nome = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[1]/div[1]/span").text
    print(nome)
    cpf = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[1]/div[2]/span/a").text
    print(cpf)
    servidor = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[1]/div[3]/span").text
    print(servidor)
    licença = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[2]/div[1]/span").text
    print(licença)
    uf = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[2]/div[2]/span").text
    print(uf)
    local_de_trabalho = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[3]/div[1]/span").text
    print(local_de_trabalho)
    ingresso_no_serviço_publico = driver.find_element_by_xpath("/html/body/main/div[2]/section[1]/div[3]/div[2]/span").text
    print(ingresso_no_serviço_publico)

    ## retira dados da ficha de remuneraçao
    try:
        remuneracao = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, 'botao-box-remuneracao'))
        )
    
    finally:
        driver.find_element_by_id('botao-box-remuneracao').click()


    try:
        remuneracao_eventual_grat_natalina = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[4]/div[2]/span[1]').text
        print(remuneracao_eventual_grat_natalina)

    except NoSuchElementException:
        remuneracao_eventual_grat_natalina = None
        print('nao tem eventual grat natalina')
    
    try:
        remuneracao_eventual_ferias = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[4]/div[2]/span[2]').text
        print(remuneracao_eventual_ferias)

    except NoSuchElementException:
        remuneracao_eventual_ferias = None
        print('nao tem eventual ferias')
    
    try:
        remuneracao_eventual_outras = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[4]/div[2]/span[3]').text
        print(remuneracao_eventual_outras)

    except NoSuchElementException:
        remuneracao_eventual_outras = None
        print('nao tem eventual outras')
    
    try:
        deduçoes_obrigatorias_irrf = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[5]/div[3]/span[1]').text
        print(deduçoes_obrigatorias_irrf)

    except NoSuchElementException:
        deduçoes_obrigatorias_irrf = None
        print('nao tem irrf')
    
    try:
        deduçoes_obrigatorias_pss_rpgs = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[5]/div[3]/span[2]').text
        print(deduçoes_obrigatorias_pss_rpgs)

    except NoSuchElementException:
        deduçoes_obrigatorias_pss_rpgs = None
        print('nao tem pss rpgs')
    
    try: 
        deduçoes_obrigatorias_pensao_militar = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[5]/div[3]/span[3]').text
        print(deduçoes_obrigatorias_pensao_militar)

    except NoSuchElementException:
        deduçoes_obrigatorias_pensao_militar = None
        print('nao tem pensao militar')
    
    try:
        deduçoes_obrigatorias_fundo_saude = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[5]/div[3]/span[4]').text
        print(deduçoes_obrigatorias_fundo_saude)

    except NoSuchElementException:
        deduçoes_obrigatorias_fundo_saude = None
        print('nao tem fundo saude')
    
    try: 
        total_remuneracao = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[6]/div[2]/strong').text
        print(total_remuneracao)

    except NoSuchElementException:
        total_remuneracao = None
        print('nao tem total remuneracao')
    
    try:
        remuneracao_basica = driver.find_element_by_xpath('//*[@id="tab-remuneracoes-1"]/div/div[2]/div[2]/span').text
        print(remuneracao_basica)

    except NoSuchElementException:
        remuneracao_basica = None
        print('nao tem remuneracao basica')
    
    ## retira dados do historico
    driver.find_element_by_id('botao-historico-poder-executivo').click()

    tabela = driver.find_element_by_id('tabela-historico-poder-executivo')

    linhas = tabela.find_element_by_tag_name('tbody').find_elements_by_tag_name('tr')

    historico_dos_vinculos = []

    for linha in range(len(linhas)):
        tdInTr = linhas[linha].find_element_by_tag_name('td')
        key_list = ['TIPO DE VINCULO', 'DATA DE INICIO DE VINCULO', 'DATA DE TERMINIO DO VINCULO',
            'ORGAO/ENTIDADE', 'CARGO/EMPREGO/FUNCAO COMISSADA'] 
        dic = {}

        for i in key_list:
            dic[i] = None
    
        for td in range(1,6): 
    
            dic[key_list[td-1]] = driver.find_element_by_xpath('//*[@id="tabela-historico-poder-executivo"]/tbody/tr['+str(linha+1)+']/td['+str(td)+']').text
    
        historico_dos_vinculos.append(json.dumps(dic))
        
    print(historico_dos_vinculos)
    
    driver.close()

    driver.switch_to.window(driver.window_handles[0])
   
       


http://www.portaltransparencia.gov.br/servidores/4806495
AARAO CARAJAS DIAS DOS SANTOS
***.182.892-**
MILITAR
NÃO

COMANDO DA AERONÁUTICA


0,00
0,00
- 5.067,65
0,00
- 834,80
- 267,12
14.843,69
16.407,55
['{"TIPO DE VINCULO": "", "DATA DE INICIO DE VINCULO": "02/03/2015", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Minist\\u00e9rio da Sa\\u00fade", "CARGO/EMPREGO/FUNCAO COMISSADA": ""}', '{"TIPO DE VINCULO": "Militares", "DATA DE INICIO DE VINCULO": "", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Comando da Aeron\\u00e1utica", "CARGO/EMPREGO/FUNCAO COMISSADA": "Primeiro-Tenente"}']
http://www.portaltransparencia.gov.br/servidores/5116961
AARAO CARLOS LUZ MACAMBIRA
***.017.623-**
CIVIL
NÃO
CEARÁ
INSTITUTO FEDERAL DO CEARÁ
11/02/2009



- 1.177,49
- 943,36
nao tem pensao militar
nao tem fundo saude
6.458,52
7.967,90
['{"TIPO DE VINCULO": "Cargo", "DATA DE INICIO DE VINCULO": "", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Instituto Federal do Cear\\u00e1", "

AARAO FERREIRA LIMA NETO
***.116.132-**
CIVIL
NÃO
PARÁ
UNIVERSIDADE FEDERAL DO PARÁ
16/08/2006
nao tem eventual grat natalina
nao tem eventual ferias
nao tem eventual outras
nao tem irrf
nao tem pss rpgs
nao tem pensao militar
nao tem fundo saude

14.249,03
['{"TIPO DE VINCULO": "Cargo", "DATA DE INICIO DE VINCULO": "13/04/2016", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Universidade Federal do Par\\u00e1", "CARGO/EMPREGO/FUNCAO COMISSADA": "PROFESSOR DO MAGISTERIO SUPERIOR"}', '{"TIPO DE VINCULO": "Cargo", "DATA DE INICIO DE VINCULO": "06/09/2012", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Universidade Federal do Par\\u00e1", "CARGO/EMPREGO/FUNCAO COMISSADA": "PROFESSOR DO MAGISTERIO SUPERIOR"}', '{"TIPO DE VINCULO": "Cargo", "DATA DE INICIO DE VINCULO": "06/09/2012", "DATA DE TERMINIO DO VINCULO": "", "ORGAO/ENTIDADE": "Universidade Federal do Par\\u00e1", "CARGO/EMPREGO/FUNCAO COMISSADA": "PROFESSOR 3 GRAU"}', '{"TIPO DE VINCULO": "Fun\\u00e7\\u00e3o", "DATA 

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="botao-box-remuneracao"]"}
  (Session info: chrome=79.0.3945.130)


In [ ]:
print